In [ ]:
pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Loading Libraries

In [ ]:
import pandas as pd
import numpy as np
import json
import gc
# import stanza
from tensorflow.keras import *
import tensorflow as tf
from tensorflow.keras import *
import tensorflow.keras.backend as K
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from transformers import TFElectraModel,ElectraTokenizer
from keras import callbacks

## Load dataset


In [ ]:
# Training
train_dataset = pd.read_csv('tsd_train.csv')

train_dataset

,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."
3,"[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]",What a knucklehead. How can anyone not know th...
4,"[32, 33, 34, 35, 36, 37, 38]","""who do you think should do the killing?""\n\nA..."
...,...,...
7934,"[8, 9, 10, 11]",Another fool pipes in.
7935,"[48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 5...",So if a restaurant owner puts up a sign saying...
7936,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",Any faith that can't stand up to logic and rea...
7937,"[5, 6, 7, 8, 9, 10, 11]",This idiotic. Use the surplus to pay down the ...


In [ ]:
# Creating python dictionary from json string for spans
train_dataset['spans'] = train_dataset['spans'].apply(lambda x : json.loads(x))
# train_dataset

In [ ]:
# Converting spans to numpy
spans = train_dataset['spans'].to_numpy()
# spans

In [ ]:
# Converting texts to numpy
texts = train_dataset['text'].to_numpy()
# texts

## Tokenize Using Electra

In [ ]:
# Tokenizer
tokenizer = ElectraTokenizer.from_pretrained('google/electra-base-discriminator')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

## Generate inputs and outputs

In [ ]:
max_length = 400
def create_outputs(texts,spans,max_length,tokenizer):
    outputs = []
    for text,span in zip(texts,spans):
        output = np.zeros(max_length*3,dtype=np.float).reshape((max_length,3))
        tokens = tokenizer.tokenize(text)[:max_length]
        length = 0
        start = True
        for i in range(len(tokens),max_length):
            output[i,0] = 1.0
        for index,token in enumerate(tokens):
            sub = False
            if "##" in token:
                sub = True
                token = token[2:]
            if not start:
                next_index = text[length:].find(token)
                if next_index == 0:
                    sub = True
                length += next_index
            # if length in span and not sub:
            #     output[index,2] = 1.0
            #     output[index,0] = 0.0
            if length in span:
                output[index,2] = 1.0
                output[index,0] = 0.0
            else:
                output[index,1] = 1.0
                output[index,0] = 0.0
            length += len(token)
            start = False
        outputs.append(output)
    return np.array(outputs)

In [ ]:
def create_inputs(texts,max_length,tokenizer):
  # Tokenize the input texts
  tokens = tokenizer(texts, max_length=max_length, padding="max_length", return_tensors="tf",truncation=True)
  # Get input length for each text
  input_length = []
  for text in texts:
    input_length.append(min(max_length,len(tokenizer.tokenize(text))))

  # Generate arrays of the tokenized inputs
  # Input IDs
  tokenized_input_ids = np.array(tokens['input_ids'])
  # Attention mask
  tokenized_token_type_ids = np.array(tokens['token_type_ids'])
  # Token type IDs
  tokenized_attention_mask = np.array(tokens['attention_mask'])
  # Input length
  nparray_input_length = np.array(input_length)
  
  inputs = [tokenized_input_ids, tokenized_token_type_ids, tokenized_attention_mask, nparray_input_length]
  return inputs

## K-Fold for Validation

In [ ]:
# # # Approximately split train:validation = 80:20
# # kf = KFold(n_splits=5)
# # train_validation_indices = []
# # for train_index, validation_index in kf.split(texts):
# #   train_validation_indices.append((train_index, validation_index))

# # train_validation_indices

# from sklearn.model_selection import train_test_split
# train_data, validation_data = train_test_split(train_dataset, test_size=0.20, random_state=42, shuffle=True)

In [ ]:
outputs = create_outputs(texts,train_dataset['spans'],max_length,tokenizer)
kf = KFold(n_splits=5)
train_validation_indices = []
for train_index,validation_index in kf.split(texts):
    train_validation_indices.append((train_index,validation_index))

<ipython-input-12-8516e645a505>:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  output = np.zeros(max_length*3,dtype=np.float).reshape((max_length,3))


## Split dataset into training and validation

In [ ]:
# # train_index, validation_index = train_validation_indices.pop()
# x_train, x_validation = list(train_dataset['text'].to_numpy()), list(validation_dataset['text'].to_numpy())
# y_train = create_outputs(train_dataset['text'].to_numpy(),train_dataset['spans'],max_length,tokenizer)
# y_validation = create_outputs(validation_dataset['text'].to_numpy(),validation_dataset['spans'],max_length,tokenizer)
# # y_train, y_validation = outputs[train_index], outputs[validation_index]
train_index, validation_index = train_validation_indices.pop()
x_train , x_validation = list(texts[train_index]) , list(texts[validation_index])
y_train , y_validation = outputs[train_index] , outputs[validation_index]

## Build Model

In [ ]:
from crf_helper import *

In [ ]:
def build_lstm_model(max_input_length,base_model):
    input_ids_layer = layers.Input(shape=(max_input_length,),name="encoder_input_ids",dtype=tf.int32)
    input_type_ids_layer = layers.Input(shape=(max_input_length,),name="encoder_token_type_ids",dtype=tf.int32)
    input_attention_mask_layer = layers.Input(shape=(max_input_length,),name="encoder_attention_mask",dtype=tf.int32)
    input_length = layers.Input(shape=(1,),name="length",dtype=tf.int32)
    base_model.trainable = True
    # a list of varying length with one or several input Tensors IN THE ORDER given in the docstring: 
    # model([input_ids, attention_mask]) or model([input_ids, attention_mask, token_type_ids])
    base_model = base_model(input_ids_layer,token_type_ids=input_type_ids_layer,attention_mask=input_attention_mask_layer,return_dict=True)
    output = layers.LSTM(512,return_sequences=True)(base_model.last_hidden_state)
    output = layers.Dense(3,activation="linear")(output)
    crf = CRFLayer()
    output = crf(inputs=[output,input_length])
    model = models.Model(inputs=[input_ids_layer,input_type_ids_layer,input_attention_mask_layer,input_length],outputs=output)
    model.compile(optimizer=optimizers.Adam(learning_rate=3e-5),loss=crf.loss,metrics=['accuracy'])
    return model

In [ ]:
def build_bilstm_model(max_input_length,base_model):
    input_ids_layer = layers.Input(shape=(max_input_length,),name="encoder_input_ids",dtype=tf.int32)
    input_type_ids_layer = layers.Input(shape=(max_input_length,),name="encoder_token_type_ids",dtype=tf.int32)
    input_attention_mask_layer = layers.Input(shape=(max_input_length,),name="encoder_attention_mask",dtype=tf.int32)
    input_length = layers.Input(shape=(1,),name="length",dtype=tf.int32)
    base_model.trainable = True
    # a list of varying length with one or several input Tensors IN THE ORDER given in the docstring: 
    # model([input_ids, attention_mask]) or model([input_ids, attention_mask, token_type_ids])
    base_model = base_model(input_ids_layer,token_type_ids=input_type_ids_layer,attention_mask=input_attention_mask_layer,return_dict=True)
    output = layers.Bidirectional(layers.LSTM(512,return_sequences=True))(base_model.last_hidden_state)
    output = layers.Dense(3,activation="linear")(output)
    crf = CRFLayer()
    output = crf(inputs=[output,input_length])
    model = models.Model(inputs=[input_ids_layer,input_type_ids_layer,input_attention_mask_layer,input_length],outputs=output)
    model.compile(optimizer=optimizers.Adam(learning_rate=3e-5),loss=crf.loss,metrics=['accuracy'])
    return model

In [ ]:
gc.collect()
tf.keras.backend.clear_session()
base_model = TFElectraModel.from_pretrained('google/electra-base-discriminator')
model_lstm = build_lstm_model(max_length,base_model)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some layers from the model checkpoint at google/electra-base-discriminator were not used when initializing TFElectraModel: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFElectraModel were initialized from the model checkpoint at google/electra-base-discriminator.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further training.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


## Fit model

In [ ]:
train_data = create_inputs(x_train,max_length,tokenizer)
validation_data = create_inputs(x_validation,max_length,tokenizer)

In [ ]:
spans_validation = spans[validation_index]
spans_train = spans[train_index]

In [ ]:
def NERGetIndicesSingleText(outputs,text,tokenizer):
    outputs = tf.argmax(outputs,axis=-1)
    tokens = tokenizer.tokenize(text)
    index = 0
    indexes = []
    sub = False
    prev = False
    for token,output in zip(tokens,outputs):
        if token[:2] == "##":
            token = token[2:]
            sub = True
        else:
            sub = False
        temp_index = text[index:].find(token)
        temp_start = index+temp_index
        if output == 2 or (sub and prev and output != 0):
            prev = True
            indexes = indexes + list(range(temp_start,temp_start+len(token)))
        else:
            prev = False
        index = temp_start+len(token)
    return np.array(indexes)

In [ ]:
def createIndicesForNERModel(predicts,texts,tokenizer):
    outputs = []
    for text,pred in zip(texts,predicts):
         indices = NERGetIndicesSingleText(pred,text,tokenizer)
         outputs.append(indices)
    return outputs

In [ ]:
def f1(preds,trues):
    if len(trues) == 0:
        return 1. if len(preds) == 0 else 0.
    if len(preds) == 0:
        return 0.
    predictions_set = set(preds)
    gold_set = set(trues)
    nom = 2 * len(predictions_set.intersection(gold_set))
    denom = len(predictions_set) + len(gold_set)
    return float(nom)/float(denom)

In [ ]:
def avg_f1(preds,trues):
    avg_f1_total = 0.0
    for pred,true in zip(preds,trues):
        avg_f1_total += f1(pred,true)
    return avg_f1_total/len(preds)

In [ ]:
class F1Metric(callbacks.Callback):
    def __init__(self,inputs,labels,spans,texts,test=True):
        self.inputs = inputs
        self.spans = spans
        self.tokenizer = tokenizer
        self.texts = texts
        self.test = test

    def on_epoch_end(self, epoch, logs={}):
        preds = self.model.predict(self.inputs,verbose=0)
        indices = createIndicesForNERModel(preds,texts,tokenizer)
        f1 = avg_f1(indices,self.spans)
        if self.test:
            print()
            print("test f1 = "+str(f1))
        else:
            print()
            print("train f1 = "+str(f1))

In [ ]:
model_lstm.fit(train_data,y_train,batch_size=16,epochs=2,callbacks=[callbacks.ModelCheckpoint("/content/drive/MyDrive/T2_Checkpoints_Giri_Surya/lstm/checkpoints/lstm",save_weights_only=True)])


Epoch 1/2
397/397 [==============================] - 687s 2s/step - loss: 9.2618 - accuracy: 0.1138
Epoch 2/2
397/397 [==============================] - 676s 2s/step - loss: 7.6286 - accuracy: 0.1124


In [ ]:
preds_lstm = model_lstm.predict(validation_data)
indices = createIndicesForNERModel(preds_lstm,x_validation,tokenizer)
f1_toxic = avg_f1(indices,spans_validation)

50/50 [==============================] - 63s 1s/step


In [ ]:
print("test F1 = %f"%(f1_toxic))

test F1 = 0.370767


## Bidirectional LSTM model

In [ ]:
gc.collect()
tf.keras.backend.clear_session()
base_model = TFElectraModel.from_pretrained('google/electra-base-discriminator')
model_bilstm = build_bilstm_model(max_length,base_model)

Some layers from the model checkpoint at google/electra-base-discriminator were not used when initializing TFElectraModel: ['discriminator_predictions']
- This IS expected if you are initializing TFElectraModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFElectraModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFElectraModel were initialized from the model checkpoint at google/electra-base-discriminator.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFElectraModel for predictions without further training.


In [ ]:
model_bilstm.fit(train_data,y_train,batch_size=8,epochs=2,callbacks=[callbacks.ModelCheckpoint("/content/drive/MyDrive/T2_Checkpoints_Giri_Surya/bilstm/checkpoints/bilstm",save_weights_only=True)])
preds_bilstm = model_bilstm.predict(validation_data)
indices = createIndicesForNERModel(preds_bilstm,x_validation,tokenizer)
f1_toxic = avg_f1(indices,spans_validation)
print("test F1 = %f"%(f1_toxic))

Epoch 1/2
794/794 [==============================] - 896s 1s/step - loss: 8.5803 - accuracy: 0.1119
Epoch 2/2
50/50 [==============================] - 66s 1s/step
test F1 = 0.404491
